In [3]:
import os, glob, pathlib

path = r"C:\Models\llm\mistral-7b-instruct-v0.1.Q8_0.gguf"
print("Exists?", os.path.exists(path))
print("Is file?", os.path.isfile(path))
print("Dir listing:", glob.glob(r"C:\models\llm\*.gguf"))
from langchain.llms import CTransformers

def create_llms_model(max_new_tokens=128, temperature=0.2, gpu_layers=8):
    # point directly to your local file path
    return CTransformers(
        model=path,
        model_type="mistral",               # important so tokenizer/arch is correct
        config={
            "max_new_tokens": max_new_tokens,
            "temperature": temperature,          # lower is more deterministic
            "gpu_layers": gpu_layers,                # CPU first; bump if you have GPU support
        },
    )
llm = create_llms_model()


Exists? True
Is file? True
Dir listing: ['C:\\models\\llm\\mistral-7b-instruct-v0.1.Q8_0.gguf']


In [ ]:
# app.py
from llama_cpp import Llama
import gradio as gr
from typing import Optional

# Path to your local GGUF model file
path = r"C:\Models\llm\mistral-7b-instruct-v0.1.Q8_0.gguf"
path = r"C:\Models\llm\Ministral-3b-instruct.Q2_K.gguf"

MODEL_PATH = path  # <- change to your filename

# Create the model client once at startup
print("Loading model... (this can take some time)")
def create_llms_model(max_new_tokens=128, temperature=0.2, gpu_layers=8):
    # point directly to your local file path
    return CTransformers(
        model=path,
        model_type="mistral",               # important so tokenizer/arch is correct
        config={
            "max_new_tokens": max_new_tokens,
            "temperature": temperature,          # lower is more deterministic
            "gpu_layers": gpu_layers,                # CPU first; bump if you have GPU support
        },
    )
llm = create_llms_model()#Llama(model_path=MODEL_PATH)
print("Model loaded.")
################

def generate(prompt: str, max_tokens: int = 256, temperature: float = 0.8, top_p: float = 0.95):
    """
    Generate text using the local model and return the generated string.
    """
    if not prompt:
        return "Please enter a prompt."
    # llm = create_llms_model(max_new_tokens=max_tokens, temperature=temperature)
    resp = llm.invoke(
        input=prompt,
        # max_tokens=max_tokens,
        # temperature=temperature,
        # top_p=top_p,
    )
    # llama-cpp-python returns {'choices':[{'text': ...}], ...}
    text = resp#.get("choices", [{}])[0].get("text", "")
    return text

# Simple Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("# Local Mistral (llama-cpp) — Gradio demo")
    with gr.Row():
        with gr.Column(scale=3):
            prompt_input = gr.Textbox(lines=6, label="Prompt")
            max_tokens = gr.Slider(minimum=16, maximum=2048, value=256, step=1, label="Max tokens")
            temperature = gr.Slider(minimum=0.0, maximum=2.0, value=0.8, step=0.01, label="Temperature")
            top_p = gr.Slider(minimum=0.0, maximum=1.0, value=0.95, step=0.01, label="Top-p")
            generate_btn = gr.Button("Generate")
        with gr.Column(scale=2):
            output = gr.Textbox(lines=12, label="Model output")

    def on_click_generate(prompt, max_tokens_val, temperature_val, top_p_val):
        return generate(prompt, max_tokens=int(max_tokens_val), temperature=float(temperature_val), top_p=float(top_p_val))

    generate_btn.click(
        on_click_generate,
        inputs=[prompt_input, max_tokens, temperature, top_p],
        outputs=[output],
    )

# Launch: runs on 7860 by default
# if __name__ == "__main__":
    # demo.launch(server_name="0.0.0.0", server_port=7860, share=False)
if __name__ == "__main__":
    demo.launch(server_name="0.0.0.0")


Loading model... (this can take some time)
Model loaded.


ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 7861): only one usage of each socket address (protocol/network address/port) is normally permitted
ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 7862): only one usage of each socket address (protocol/network address/port) is normally permitted
ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 7863): only one usage of each socket address (protocol/network address/port) is normally permitted
ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 7864): only one usage of each socket address (protocol/network address/port) is normally permitted
ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 7865): only one usage of each socket address (protocol/network address/port) is normally permitted


* Running on local URL:  http://0.0.0.0:7866
* To create a public link, set `share=True` in `launch()`.
